In [1]:
#import necessary libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
from scipy import stats
from scipy.stats import norm, skew #for some statistics

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
#import data
path=('F:/Dropbox/Columbia MSBA/IEOR E4501 Tools for Analytics/Project/airbnb-price-prediction/train.csv')
data=pd.read_csv(path)

In [ ]:
data.head()

In [ ]:
print (len(data))

In [ ]:
data.columns

In [ ]:
len(data.columns)

In [ ]:
#if one column only has one value, consider to delete the column
columns_to_remove = [col for col in data.columns if data[col].nunique() == 1]

In [ ]:
print(columns_to_remove)

In [ ]:
for col in data.columns:
    if data[col].isnull().sum() > 0:
        rate = data[col].isnull().sum() * 100 / data.shape[0]
        print(f'Column {col} has {rate:.4f}% missing values.')
    if data[col].dtype == 'object':
        if (data[col] == 'not available in demo dataset').sum() > 0:
            rate = (data[col] == 'not available in demo dataset').sum() * 100 / data.shape[0]
            print(f'Column {col} has {rate:.4f}% values not available in dataset.')

In [ ]:
# deal with dates
import time
import datetime
def dif_time(date):
    now_time = datetime.datetime.now()
    now_time = now_time.strftime('%Y-%m-%d')
    d2 = datetime.datetime.strptime(now_time, '%Y-%m-%d')
    if str(date) != 'nan':
        d1 = datetime.datetime.strptime(str(date), '%Y-%m-%d')
        day = (d2 - d1).days
        return day
    else:
        return None

In [ ]:
data['last_review'] = data['last_review'].apply(dif_time)

In [ ]:
data['neighbourhood'].nunique()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
for col in train_data.columns:
    print(col)
    lbl = LabelEncoder()
    lbl.fit(list(train_data[col].values.astype('str')) + list(test_data[col].values.astype('str')))
    train_data[col] = lbl.transform(list(train_data[col].values.astype('str')))
    test_data[col] = lbl.transform(list(test_data[col].values.astype('str')))

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
from sklearn.model_selection import KFold #Provides train/test indices to split data in train/test sets. Split dataset into k consecutive folds (without shuffling by default).
import lightgbm as lgb

In [ ]:
params = {"objective" : "regression", "metric" : "rmse", "max_depth": 8, "min_child_samples": 20, "reg_alpha": 0.2, "reg_lambda": 0.2,
        "num_leaves" : 257, "learning_rate" : 0.01, "subsample" : 0.9, "colsample_bytree" : 0.9, "subsample_freq ": 5}
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=False, random_state=42)
# Cleaning and defining parameters for LGBM
model = lgb.LGBMRegressor(**params, n_estimators = 20000, nthread = 4, n_jobs = -1)

In [ ]:
prediction = np.zeros(test_data.shape[0])

for fold_n, (train_index, test_index) in enumerate(folds.split(X)):
    print('Fold:', fold_n)
    #print(f'Train samples: {len(train_index)}. Valid samples: {len(test_index)}')
    X_train, X_valid = X.iloc[train_index], X.iloc[test_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
    

    model.fit(X_train, y_train, 
            eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='rmse',
            verbose=500, early_stopping_rounds=10)
    
    y_pred = model.predict(X_test, num_iteration=model.best_iteration_)
    prediction += y_pred
prediction /= n_fold